# Data Wrangling & Exploration

This notebook captueres the data wrangling and exploration of image stabalization data sets. We have accress to multiple data sets.  They have been gathered to insure quicker access to the original data sets that are distriputed across the internet. It also means the data is under a type of source control, frozen at the version I pulled. This is both good and bad because it will be dificult to see modifications that I may want. But we will cross that bridge if it comes up.

## Read the StabNet Data Set

A copy of the StabNet Data set is frozen on an S3.  I will read it from there extract the data.

In [1]:
%%time

import os
import boto3

project_dir = os.getcwd()

data_dir = 'Data'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)
os.chdir(data_dir)

s3 = boto3.client('s3')
s3.download_file('madat-machine-learning-data', 'Mark/capstone-project/DeepStab.zip', 'DeepStab.zip')

from zipfile import ZipFile

with ZipFile('DeepStab.zip', 'r') as zipObj:
    zipObj.extractall()
    
os.chdir('..')
os.getcwd()

CPU times: user 1min 5s, sys: 36.7 s, total: 1min 42s
Wall time: 3min 17s


'/home/sagemaker-user/hand-tracking-stabilization'

From the Readme.txt:

Here we provide the DeepStab dataset of the pre-print paper 'Deep Online Video Stabilization'.
The dataset contains 1280x720 paired videos, from each pair one is stable and the other is unstable.
The videos are organized in two corresponding folders ./stable and ./unstable

In [2]:
import os.path

stable_avis = 'Data/DeepStab/stable'
unstable_avis = 'Data/DeepStab/unstable'
number_stable_avis = len([name for name in os.listdir(stable_avis) if os.path.isfile(os.path.join(stable_avis, name))])
number_unstable_avis = len([name for name in os.listdir(unstable_avis) if os.path.isfile(os.path.join(unstable_avis, name))])

number_stable_avis , number_unstable_avis

(61, 61)

In [3]:
### make sure libGL is on the instance
!/opt/conda/bin/python -m pip install --upgrade pip
!pip install opencv-python

!sudo apt update
!sudo apt -y install libgl1-mesa-glx

  Using cached pip-21.2.4-py3-none-any.whl (1.6 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.2.2
    Uninstalling pip-21.2.2:
      Successfully uninstalled pip-21.2.2
Defaulting to user installation because normal site-packages is not writeable
sudo: unable to resolve host sagemaker-jumpstart-ml-g4dn-xlarge-615c7d2695dab5c6f7e6dd5c954b
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease   
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1424 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:6 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2294 kB]33m
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2730 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Pac

Verify the stable and unstable frame counts match and that they are of the correct shape.

In [4]:
%%time

import cv2
import os

stable_avi_frame_count = 0
for file_number in range(1,number_stable_avis+1):
           
    vidcap = cv2.VideoCapture(os.path.join(stable_avis, f'{file_number}.avi'))
    success,image = vidcap.read()

    while success:
        if image.shape != (720,1280,3):
            print(f'Invalid Image shape: {image.shape}')
        stable_avi_frame_count += 1
                  
        success,image = vidcap.read()

unstable_avi_frame_count = 0
for file_number in range(1,number_unstable_avis+1):
           
    vidcap = cv2.VideoCapture(os.path.join(unstable_avis, f'{file_number}.avi'))
    success,image = vidcap.read()

    while success:
        if image.shape != (720,1280,3):
            print(f'Invalid Image shape: {image.shape}')
        unstable_avi_frame_count += 1
     
        success,image = vidcap.read()
        
stable_avi_frame_count, unstable_avi_frame_count
               

CPU times: user 4min 50s, sys: 2.85 s, total: 4min 53s
Wall time: 4min 57s


(43585, 43585)

Note: The image shapes are consistent (720,1280,3).

## Read the Dataset used for Evaluation Deep Learning Based Video Stabilization System Frameworks

An important aspect of this data set is it contains translation data for the labeled stabel data set. This is seen as extremely useful for understanding and evaluating the performance of the model.

This is going to take some time. Tar is extreamely slow due to its serial nature and compression. The data set load is in the order of 55 minutes. 


In [5]:
%%time

import os
import boto3
import tarfile

os.chdir(project_dir)
print(os.getcwd())

data_dir = 'Data/EvaluationDataSet'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)
os.chdir(data_dir)

s3 = boto3.client('s3')
s3.download_file('madat-machine-learning-data', 'Mark/capstone-project/Dataset.tar.gz', 'DataSet.tar.gz')
s3.download_file('madat-machine-learning-data', 'Mark/capstone-project/Dataset-3.tar.gz', 'DataSet-3.tar.gz')

tar = tarfile.open('DataSet.tar.gz')
tar.extractall()
tar.close()

tar = tarfile.open('DataSet-3.tar.gz')
tar.extractall()
tar.close()

os.chdir('../..')
os.getcwd()

/home/sagemaker-user/hand-tracking-stabilization
CPU times: user 2min 28s, sys: 2min 12s, total: 4min 40s
Wall time: 58min 42s


'/home/sagemaker-user/hand-tracking-stabilization'

Determine the number of unlabeled and labeled records.

The dataset contains 910x512 paired videos. In this data set each video is in its own numbered directory. The unstable video (unstable.mp4) and the stable is (GT.mp4) both are found in the directory.

<left><img src="img/evaluation-videos.JPG" width=250 /></left>
                           
This data set also contains trasitional and rotation transitions associated with the videos under the video number.

<left><img src="img/rotation-translational-files.JPG" width=250 /></left>


In [6]:
import os.path

evaluation_dataset_location = 'Data/EvaluationDataSet'
sample_ids = []
for possible_sample_ids in os.listdir(evaluation_dataset_location):
    full_name = os.path.join(evaluation_dataset_location, possible_sample_ids)
    if os.path.isdir(full_name):
        sample_ids.append(full_name)

sample_ids.sort()
len(sample_ids)

311

The folowing is a non-trivial analysis of the mp4's in the evaluation data set.

It takes a significant amount of time to do the frame analysis.

In [7]:
%%time

import cv2
import os
import pandas

def is_video_valid(video_being_processed):
    vidcap = cv2.VideoCapture(video_being_processed)
    success,image = vidcap.read()
    return success

def get_frame_count(video_being_processed):
    print(f'video_being_processed: {video_being_processed}')
    vidcap = cv2.VideoCapture(video_being_processed)
    success,image = vidcap.read()
    if success: 
        unstable_video_count += 1

data = []

for sample_id in sample_ids:
    unstable_file_name = os.path.join(sample_id, 'unstable.mp4')
    stable_file_name = os.path.join(sample_id, 'GT.mp4')
    transition_file_name = os.path.join(sample_id, 'PR', 'rot_tra', 'translation.txt')
    rotation_file_name = os.path.join(sample_id, 'PR', 'rot_tra', 'rotation.txt')
    
    unstable_frame_count_by_analysis = 0
    
    
    data.append({
        'video_number' : os.path.basename(sample_id),
        'unstable_file_name': unstable_file_name,
        'unstable_file_is_valid': is_video_valid(unstable_file_name),
        'stable_file_name': stable_file_name,
        'stable_file_is_valid': is_video_valid(stable_file_name),
        'transition_file_name': transition_file_name,
        'rotation_file_name': rotation_file_name
    })

df = pandas.DataFrame(data, columns=['video_number', 'unstable_file_name', 'unstable_file_is_valid', 'stable_file_name', 'stable_file_is_valid', 'transition_file_name','rotation_file_name'])
df

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x5593678d7080] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x5593678eb0c0] moov atom not found


CPU times: user 4min 3s, sys: 7.39 s, total: 4min 10s
Wall time: 2min 24s


,video_number,unstable_file_name,unstable_file_is_valid,stable_file_name,stable_file_is_valid,transition_file_name,rotation_file_name
0,00001,Data/EvaluationDataSet/00001/unstable.mp4,True,Data/EvaluationDataSet/00001/GT.mp4,True,Data/EvaluationDataSet/00001/PR/rot_tra/transl...,Data/EvaluationDataSet/00001/PR/rot_tra/rotati...
1,00002,Data/EvaluationDataSet/00002/unstable.mp4,True,Data/EvaluationDataSet/00002/GT.mp4,True,Data/EvaluationDataSet/00002/PR/rot_tra/transl...,Data/EvaluationDataSet/00002/PR/rot_tra/rotati...
2,00003,Data/EvaluationDataSet/00003/unstable.mp4,True,Data/EvaluationDataSet/00003/GT.mp4,True,Data/EvaluationDataSet/00003/PR/rot_tra/transl...,Data/EvaluationDataSet/00003/PR/rot_tra/rotati...
3,00004,Data/EvaluationDataSet/00004/unstable.mp4,True,Data/EvaluationDataSet/00004/GT.mp4,True,Data/EvaluationDataSet/00004/PR/rot_tra/transl...,Data/EvaluationDataSet/00004/PR/rot_tra/rotati...
4,00005,Data/EvaluationDataSet/00005/unstable.mp4,True,Data/EvaluationDataSet/00005/GT.mp4,True,Data/EvaluationDataSet/00005/PR/rot_tra/transl...,Data/EvaluationDataSet/00005/PR/rot_tra/rotati...
...,...,...,...,...,...,...,...
306,00417,Data/EvaluationDataSet/00417/unstable.mp4,True,Data/EvaluationDataSet/00417/GT.mp4,True,Data/EvaluationDataSet/00417/PR/rot_tra/transl...,Data/EvaluationDataSet/00417/PR/rot_tra/rotati...
307,00418,Data/EvaluationDataSet/00418/unstable.mp4,True,Data/EvaluationDataSet/00418/GT.mp4,True,Data/EvaluationDataSet/00418/PR/rot_tra/transl...,Data/EvaluationDataSet/00418/PR/rot_tra/rotati...
308,00419,Data/EvaluationDataSet/00419/unstable.mp4,True,Data/EvaluationDataSet/00419/GT.mp4,True,Data/EvaluationDataSet/00419/PR/rot_tra/transl...,Data/EvaluationDataSet/00419/PR/rot_tra/rotati...
309,00420,Data/EvaluationDataSet/00420/unstable.mp4,True,Data/EvaluationDataSet/00420/GT.mp4,True,Data/EvaluationDataSet/00420/PR/rot_tra/transl...,Data/EvaluationDataSet/00420/PR/rot_tra/rotati...


Clean up the samples before digging into the individual frames

In [8]:
good_samples = df[(df['unstable_file_is_valid']) & (df['stable_file_is_valid']) ]
good_samples

,video_number,unstable_file_name,unstable_file_is_valid,stable_file_name,stable_file_is_valid,transition_file_name,rotation_file_name
0,00001,Data/EvaluationDataSet/00001/unstable.mp4,True,Data/EvaluationDataSet/00001/GT.mp4,True,Data/EvaluationDataSet/00001/PR/rot_tra/transl...,Data/EvaluationDataSet/00001/PR/rot_tra/rotati...
1,00002,Data/EvaluationDataSet/00002/unstable.mp4,True,Data/EvaluationDataSet/00002/GT.mp4,True,Data/EvaluationDataSet/00002/PR/rot_tra/transl...,Data/EvaluationDataSet/00002/PR/rot_tra/rotati...
2,00003,Data/EvaluationDataSet/00003/unstable.mp4,True,Data/EvaluationDataSet/00003/GT.mp4,True,Data/EvaluationDataSet/00003/PR/rot_tra/transl...,Data/EvaluationDataSet/00003/PR/rot_tra/rotati...
3,00004,Data/EvaluationDataSet/00004/unstable.mp4,True,Data/EvaluationDataSet/00004/GT.mp4,True,Data/EvaluationDataSet/00004/PR/rot_tra/transl...,Data/EvaluationDataSet/00004/PR/rot_tra/rotati...
4,00005,Data/EvaluationDataSet/00005/unstable.mp4,True,Data/EvaluationDataSet/00005/GT.mp4,True,Data/EvaluationDataSet/00005/PR/rot_tra/transl...,Data/EvaluationDataSet/00005/PR/rot_tra/rotati...
...,...,...,...,...,...,...,...
306,00417,Data/EvaluationDataSet/00417/unstable.mp4,True,Data/EvaluationDataSet/00417/GT.mp4,True,Data/EvaluationDataSet/00417/PR/rot_tra/transl...,Data/EvaluationDataSet/00417/PR/rot_tra/rotati...
307,00418,Data/EvaluationDataSet/00418/unstable.mp4,True,Data/EvaluationDataSet/00418/GT.mp4,True,Data/EvaluationDataSet/00418/PR/rot_tra/transl...,Data/EvaluationDataSet/00418/PR/rot_tra/rotati...
308,00419,Data/EvaluationDataSet/00419/unstable.mp4,True,Data/EvaluationDataSet/00419/GT.mp4,True,Data/EvaluationDataSet/00419/PR/rot_tra/transl...,Data/EvaluationDataSet/00419/PR/rot_tra/rotati...
309,00420,Data/EvaluationDataSet/00420/unstable.mp4,True,Data/EvaluationDataSet/00420/GT.mp4,True,Data/EvaluationDataSet/00420/PR/rot_tra/transl...,Data/EvaluationDataSet/00420/PR/rot_tra/rotati...


Loop through the videos that we know we can read to build up the processing information. This includes the shape of images in the video clip and the labeled transition per frame data. This information is needed if we use the data set and the labeled transitions. This table will alo identify the total number of frames that are being used to train a future model. The shape is relavent so that the size could be normalized to the models resolution.

This is going to take almost an hour to process the data. 

Note: In getting the algorithm to work, I used a smaller data set (Just the first video) to insure the format was correct.

In [9]:
%%time

frames = pandas.DataFrame()

for index, row in good_samples.iterrows():
    #if int(row['video_number']) > 1 : 
    #    break 

    unstable_video_being_processed = row['unstable_file_name']
    print(f'video_being_processed: {unstable_video_being_processed}')
    vidcap = cv2.VideoCapture(unstable_video_being_processed)
    success,image = vidcap.read()
    
    if success:
        transitions = pandas.read_csv(row['transition_file_name'], header=None, delimiter=r"\s+")
        rotations = pandas.read_csv(row['rotation_file_name'], header=None, delimiter=r"\s+")
    
    frame_number = 0
    while success:
        row['frame_number'] = frame_number
        row['image_hieght'] = image.shape[0]
        row['image_width'] = image.shape[1]
        row['image_channels'] = image.shape[2]
        row['t1'] = transitions.loc[frame_number][0]
        row['t2'] = transitions.loc[frame_number][1]
        row['t3'] = transitions.loc[frame_number][2]
        row['r1'] = rotations.loc[frame_number][0]
        row['r2'] = rotations.loc[frame_number][1]
        row['r3'] = rotations.loc[frame_number][2]
        
        frames = frames.append(row)
        frame_number +=1
        
        success,image = vidcap.read()
        
frames


video_being_processed: Data/EvaluationDataSet/00001/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00002/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00003/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00004/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00005/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00006/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00007/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00008/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00009/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00010/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00011/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00012/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00013/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00014/unstable.mp4
video_being_processed: Data/EvaluationDataSet/00015/unstable.mp4
video_being_processed: Da

,video_number,unstable_file_name,unstable_file_is_valid,stable_file_name,stable_file_is_valid,transition_file_name,rotation_file_name,frame_number,image_hieght,image_width,image_channels,t1,t2,t3,r1,r2,r3
0,00001,Data/EvaluationDataSet/00001/unstable.mp4,1.0,Data/EvaluationDataSet/00001/GT.mp4,1.0,Data/EvaluationDataSet/00001/PR/rot_tra/transl...,Data/EvaluationDataSet/00001/PR/rot_tra/rotati...,0.0,512.0,910.0,3.0,-3.83,3.12,-1.08,0.01,0.01,0.01
0,00001,Data/EvaluationDataSet/00001/unstable.mp4,True,Data/EvaluationDataSet/00001/GT.mp4,True,Data/EvaluationDataSet/00001/PR/rot_tra/transl...,Data/EvaluationDataSet/00001/PR/rot_tra/rotati...,1.0,512.0,910.0,3.0,0.52,8.91,3.86,0.00,0.00,0.00
0,00001,Data/EvaluationDataSet/00001/unstable.mp4,True,Data/EvaluationDataSet/00001/GT.mp4,True,Data/EvaluationDataSet/00001/PR/rot_tra/transl...,Data/EvaluationDataSet/00001/PR/rot_tra/rotati...,2.0,512.0,910.0,3.0,2.37,11.38,5.96,0.00,0.00,0.00
0,00001,Data/EvaluationDataSet/00001/unstable.mp4,True,Data/EvaluationDataSet/00001/GT.mp4,True,Data/EvaluationDataSet/00001/PR/rot_tra/transl...,Data/EvaluationDataSet/00001/PR/rot_tra/rotati...,3.0,512.0,910.0,3.0,2.14,11.08,5.70,-0.00,-0.00,-0.00
0,00001,Data/EvaluationDataSet/00001/unstable.mp4,True,Data/EvaluationDataSet/00001/GT.mp4,True,Data/EvaluationDataSet/00001/PR/rot_tra/transl...,Data/EvaluationDataSet/00001/PR/rot_tra/rotati...,4.0,512.0,910.0,3.0,0.26,8.57,3.56,-0.00,-0.00,-0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,00421,Data/EvaluationDataSet/00421/unstable.mp4,True,Data/EvaluationDataSet/00421/GT.mp4,True,Data/EvaluationDataSet/00421/PR/rot_tra/transl...,Data/EvaluationDataSet/00421/PR/rot_tra/rotati...,95.0,576.0,1024.0,3.0,13.25,15.47,13.16,-0.01,0.00,-0.00
310,00421,Data/EvaluationDataSet/00421/unstable.mp4,True,Data/EvaluationDataSet/00421/GT.mp4,True,Data/EvaluationDataSet/00421/PR/rot_tra/transl...,Data/EvaluationDataSet/00421/PR/rot_tra/rotati...,96.0,576.0,1024.0,3.0,12.60,14.68,12.46,-0.01,0.00,-0.00
310,00421,Data/EvaluationDataSet/00421/unstable.mp4,True,Data/EvaluationDataSet/00421/GT.mp4,True,Data/EvaluationDataSet/00421/PR/rot_tra/transl...,Data/EvaluationDataSet/00421/PR/rot_tra/rotati...,97.0,576.0,1024.0,3.0,12.58,14.65,12.44,-0.01,0.00,-0.00
310,00421,Data/EvaluationDataSet/00421/unstable.mp4,True,Data/EvaluationDataSet/00421/GT.mp4,True,Data/EvaluationDataSet/00421/PR/rot_tra/transl...,Data/EvaluationDataSet/00421/PR/rot_tra/rotati...,98.0,576.0,1024.0,3.0,13.41,15.67,13.34,-0.01,-0.00,-0.00


In [10]:
frames.to_csv('data-evaluation-data-set.csv')

## Lets Analyze the Dataset used for Evaluation Deep Learning Based Video Stabilization System Frameworks


In [11]:
frames[['image_hieght','image_width','image_channels','t1','t2','t3','r1','r2','r3']].describe()

,image_hieght,image_width,image_channels,t1,t2,t3,r1,r2,r3
count,102827.000000,102827.000000,102827.0,102827.000000,102827.000000,102827.000000,102827.000000,102827.000000,102827.000000
mean,1160.078112,2008.443152,3.0,-0.020806,0.109123,0.091565,0.001280,0.001259,0.001235
std,470.586140,835.159589,0.0,11.900900,11.928121,11.092596,0.007255,0.007240,0.006426
min,288.000000,512.000000,3.0,-41.690000,-43.610000,-43.040000,-0.020000,-0.020000,-0.020000
25%,864.000000,1536.000000,3.0,-8.430000,-8.390000,-7.140000,0.000000,0.000000,0.000000
50%,864.000000,1536.000000,3.0,-0.110000,0.120000,-0.000000,0.000000,-0.000000,0.000000
75%,1728.000000,3072.000000,3.0,8.480000,8.610000,7.310000,0.010000,0.010000,0.010000
max,3072.000000,3278.000000,3.0,37.320000,38.970000,36.780000,0.020000,0.030000,0.020000


So we can see:

* 102,827 video frames for training.
* Videos in the data set have different size images. That means we willneed to normalize the image size before it is sent to any model.
* Rotation is small. These videos would not be good for training a model used for a large degree of camera rotation.